## Basic Questions

### 1. Create a Flask app that displays "Hello, World!" on the homepage:

In [ ]:
from flask import Flask

app = Flask(__name__)

@app.route('/')
def hello_world():
    return 'Hello, World!'

if __name__ == '__main__':
    app.run()

### 2. Build a Flask app with static HTML pages and navigate between them:

In [ ]:
from flask import Flask, render_template

app = Flask(__name__)

@app.route('/')
def home():
    return render_template('index.html')

@app.route('/about')
def about():
    return render_template('index.html')

if __name__ == '__main__':
    app.run()

In [ ]:
# basic html code of index.html 
<!DOCTYPE html>
<html>

<head>
    <title>Form Example</title>
</head>

<body>
    <h1>Basic html webpage</h1>
    
</body>

</html>

### 3. Develop a Flask app that uses URL parameters to display dynamic content:

In [ ]:
from flask import Flask

app = Flask(__name__)

@app.route('/greet/<name>')
def greet(name):
    return f'Hello, {name}!'

if __name__ == '__main__':
    app.run() 

### 4. Create a Flask app with a form that accepts user input and displays it:

In [ ]:
from flask import Flask,request,jsonify,render_template 

app = Flask("__name__") 

@app.route("/")
def user_form():
    return render_template("index.html") 

@app.route("/form_result",methods=['POST'])
def inputs():
    name = request.form.get("username")
    return f'{name}'


if __name__=="__main__":
    app.run(host="0.0.0.0")

In [ ]:
# index.html file in template folder

<!DOCTYPE html>
<html>

<head>
    <title>Form Example</title>
</head>

<body>
    <h1>Submit a Form</h1>
    <form action="/form_result" method="post">
        <label for="username">Username:</label>
        <input type="text" id="username" name="username" required><br>
        <input type="submit" value="ok">
    </form>
</body>

</html>

### 5. Implement user sessions in a Flask app to store and display user-specific data:

In [ ]:
from flask import Flask, request, session, redirect, url_for, render_template
from flask_session import Session

app = Flask(__name__)
app.config['SECRET_KEY'] = 'mysecretkey'
app.config['SESSION_TYPE'] = 'filesystem'
Session(app)

@app.route('/')
def index():
    return 'Welcome to the Flask Session Example'

@app.route('/set_session/<value>')
def set_session(value):
    session['user_value'] = value
    return 'Session value set'

@app.route('/get_session')
def get_session():
    user_value = session.get('user_value', 'Session not set')
    return f'Session Value: {user_value}'

if __name__ == '__main__':
    app.run(host="0.0.0.0") 

## Intermediate Flask Topics:

### Build a Flask app that allows users to upload files and display them on the website.

In [ ]:
from flask import Flask, render_template, request, redirect, url_for, send_from_directory 
from werkzeug.utils import secure_filename
from werkzeug.datastructures import FileStorage 
from flask_uploads import UploadSet, configure_uploads, TEXT, DOCUMENTS 

app = Flask(__name__)
app.config['UPLOADED_FILES_DEST'] = 'uploads'  # Destination folder for uploaded files
files = UploadSet('files', TEXT + DOCUMENTS)
configure_uploads(app, files)

@app.route('/')
def index():
    return render_template('index.html')

@app.route('/upload', methods=['POST'])
def upload():
    if 'file' in request.files:
        file = request.files['file']
        if file:
            filename = files.save(file)
            return f'File "{filename}" uploaded successfully!'
    return 'No file provided.'

@app.route('/uploads/<filename>')
def uploaded_file(filename):
    return send_from_directory(app.config['UPLOADED_FILES_DEST'], filename)

if __name__ == '__main__':
    app.run(host="0.0.0.0")

In [ ]:
### Code of index.html file

In [ ]:
<!DOCTYPE html>
<html>
<head>
    <title>File Upload</title>
</head>
<body>
    <h1>File Upload</h1>
    <form method="POST" action="/upload" enctype="multipart/form-data">
        <select name="file_type">
            <option value="files">Text</option>
            <option value="photos">Image</option>
            <option value="pdfs">PDF</option>
        </select>
        <input type="file" name="file">
        <input type="submit" value="Upload">
    </form>
</body>
</html>


### Integrate a SQLite database with Flask to perform CRUD operations on a list of items.

In [ ]:
# Set up Flask and SQLite.

from flask import Flask, render_template, request, redirect, url_for
from flask_sqlalchemy import SQLAlchemy

app = Flask(__name__)
app.config['SQLALCHEMY_DATABASE_URI'] = 'sqlite:///items.db'
db = SQLAlchemy(app)


In [ ]:
# Define a model for your items.
class Item(db.Model):
    id = db.Column(db.Integer, primary_key=True)
    name = db.Column(db.String(100), nullable=False)

    def __repr__(self):
        return f"Item('{self.name}')"


In [ ]:
# Create routes for CRUD operations.

@app.route('/')
def index():
    items = Item.query.all()
    return render_template('index.html', items=items)

@app.route('/add', methods=['POST'])
def add_item():
    name = request.form['name']
    new_item = Item(name=name)
    db.session.add(new_item)
    db.session.commit()
    return redirect(url_for('index'))

@app.route('/delete/<int:id>')
def delete_item(id):
    item_to_delete = Item.query.get_or_404(id)
    db.session.delete(item_to_delete)
    db.session.commit()
    return redirect(url_for('index'))


In [ ]:
# Implement HTML templates for user interaction. IN templates folder

<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>Items</title>
</head>
<body>
    <h1>Items</h1>
    <form action="/add" method="post">
        <input type="text" name="name" placeholder="Enter item name">
        <button type="submit">Add Item</button>
    </form>
    <ul>
        {% for item in items %}
            <li>{{ item.name }} <a href="/delete/{{ item.id }}">Delete</a></li>
        {% endfor %}
    </ul>
</body>
</html>


### Implement user authentication and registration in a Flask app using Flask-Login.

In [ ]:
# Create a Flask app and set up the necessary configurations:

from flask import Flask
from flask_sqlalchemy import SQLAlchemy
from flask_login import LoginManager

app = Flask(__name__)
app.config['SECRET_KEY'] = 'your_secret_key'  # Change this to your own secret key
app.config['SQLALCHEMY_DATABASE_URI'] = 'sqlite:///site.db'  # Change this to your own database URI

db = SQLAlchemy(app)
login_manager = LoginManager(app)
login_manager.login_view = 'login'

In [ ]:
# Create a User model for storing user information:

from flask_login import UserMixin

class User(db.Model, UserMixin):
    id = db.Column(db.Integer, primary_key=True)
    username = db.Column(db.String(20), unique=True, nullable=False)
    password = db.Column(db.String(60), nullable=False)

In [ ]:
# Create routes for user authentication and registration:

from flask import render_template, redirect, request, flash
from werkzeug.security import generate_password_hash, check_password_hash
from flask_login import login_user, logout_user, login_required

@app.route('/register', methods=['GET', 'POST'])
def register():
    if request.method == 'POST':
        username = request.form.get('username')
        password = request.form.get('password')
        hashed_password = generate_password_hash(password)
        new_user = User(username=username, password=hashed_password)
        db.session.add(new_user)
        db.session.commit()
        flash('Your account has been created! You can now log in.', 'success')
        return redirect('/login')
    return render_template('register.html')

@app.route('/login', methods=['GET', 'POST'])
def login():
    if request.method == 'POST':
        username = request.form.get('username')
        password = request.form.get('password')
        user = User.query.filter_by(username=username).first()
        if user and check_password_hash(user.password, password):
            login_user(user)
            next_page = request.args.get('next')
            return redirect(next_page) if next_page else redirect('/')
        else:
            flash('Login unsuccessful. Please check your username and password.', 'danger')
    return render_template('login.html')

@app.route('/logout')
@login_required
def logout():
    logout_user()
    return redirect('/')


In [ ]:
# register.html 

<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>Register</title>
</head>
<body>
    <h1>Register</h1>
    {% with messages = get_flashed_messages(with_categories=true) %}
        {% if messages %}
            {% for category, message in messages %}
                <div class="alert alert-{{ category }}">{{ message }}</div>
            {% endfor %}
        {% endif %}
    {% endwith %}
    <form action="/register" method="post">
        <div>
            <label for="username">Username:</label>
            <input type="text" id="username" name="username" required>
        </div>
        <div>
            <label for="password">Password:</label>
            <input type="password" id="password" name="password" required>
        </div>
        <button type="submit">Register</button>
    </form>
    <p>Already have an account? <a href="/login">Login</a></p>
</body>
</html>


In [ ]:
# login.html 

<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>Login</title>
</head>
<body>
    <h1>Login</h1>
    {% with messages = get_flashed_messages(with_categories=true) %}
        {% if messages %}
            {% for category, message in messages %}
                <div class="alert alert-{{ category }}">{{ message }}</div>
            {% endfor %}
        {% endif %}
    {% endwith %}
    <form action="/login" method="post">
        <div>
            <label for="username">Username:</label>
            <input type="text" id="username" name="username" required>
        </div>
        <div>
            <label for="password">Password:</label>
            <input type="password" id="password" name="password" required>
        </div>
        <button type="submit">Login</button>
    </form>
    <p>Don't have an account yet? <a href="/register">Register</a></p>
</body>
</html>


### Create a RESTful API using Flask to perform CRUD operations on resources like books or movies.

In [ ]:
from flask import Flask, request, jsonify

app = Flask(__name__)

# Sample data to simulate a database
books = [
    {"id": 1, "title": "Harry Potter", "author": "J.K. Rowling"},
    {"id": 2, "title": "Lord of the Rings", "author": "J.R.R. Tolkien"}
]
next_id = 3  # Initial ID for new books


# Get all books
@app.route('/books', methods=['GET'])
def get_books():
    return jsonify(books)


# Get a single book by ID
@app.route('/books/<int:book_id>', methods=['GET'])
def get_book(book_id):
    book = next((book for book in books if book['id'] == book_id), None)
    if book:
        return jsonify(book)
    else:
        return jsonify({"error": "Book not found"}), 404


# Add a new book
@app.route('/books', methods=['POST'])
def add_book():
    global next_id
    data = request.json
    new_book = {"id": next_id, "title": data['title'], "author": data['author']}
    books.append(new_book)
    next_id += 1
    return jsonify(new_book), 201


# Update a book
@app.route('/books/<int:book_id>', methods=['PUT'])
def update_book(book_id):
    book = next((book for book in books if book['id'] == book_id), None)
    if not book:
        return jsonify({"error": "Book not found"}), 404
    data = request.json
    book['title'] = data.get('title', book['title'])
    book['author'] = data.get('author', book['author'])
    return jsonify(book)


# Delete a book
@app.route('/books/<int:book_id>', methods=['DELETE'])
def delete_book(book_id):
    global books
    books = [book for book in books if book['id'] != book_id]
    return jsonify({"message": "Book deleted successfully"}), 200


if __name__ == '__main__':
    app.run(debug=True)

### Design a Flask app with proper error handling for 404 and 500 errors.

In [ ]:
from flask import Flask, jsonify

app = Flask(__name__)

# Sample data to simulate a database
books = [
    {"id": 1, "title": "Harry Potter", "author": "J.K. Rowling"},
    {"id": 2, "title": "Lord of the Rings", "author": "J.R.R. Tolkien"}
]
next_id = 3  # Initial ID for new books


# Custom error handlers
@app.errorhandler(404)
def not_found_error(error):
    return jsonify({"error": "Not found"}), 404


@app.errorhandler(500)
def internal_error(error):
    return jsonify({"error": "Internal server error"}), 500


# Routes for CRUD operations on books
@app.route('/books', methods=['GET'])
def get_books():
    return jsonify(books)


@app.route('/books/<int:book_id>', methods=['GET'])
def get_book(book_id):
    book = next((book for book in books if book['id'] == book_id), None)
    if book:
        return jsonify(book)
    else:
        return not_found_error(404)


@app.route('/books', methods=['POST'])
def add_book():
    global next_id
    data = request.json
    new_book = {"id": next_id, "title": data['title'], "author": data['author']}
    books.append(new_book)
    next_id += 1
    return jsonify(new_book), 201


@app.route('/books/<int:book_id>', methods=['PUT'])
def update_book(book_id):
    book = next((book for book in books if book['id'] == book_id), None)
    if not book:
        return not_found_error(404)
    data = request.json
    book['title'] = data.get('title', book['title'])
    book['author'] = data.get('author', book['author'])
    return jsonify(book)


@app.route('/books/<int:book_id>', methods=['DELETE'])
def delete_book(book_id):
    global books
    books = [book for book in books if book['id'] != book_id]
    return jsonify({"message": "Book deleted successfully"}), 200


if __name__ == '__main__':
    app.run(debug=True)
